In [175]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from dummyPy import OneHotEncoder
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics
import matplotlib.pyplot as plt
#
from xgboost import XGBClassifier


In [176]:
train=pd.read_csv('./data/train_users_2.csv', encoding='utf-8')
test=pd.read_csv('./data/test_users.csv', encoding='utf-8')
dataset=pd.read_csv('./data/train_users_2.csv', encoding='utf-8')
labels=train['country_destination']

In [177]:
df_all=train.append(test)

C:\Python\Anaconda3-5.2\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [178]:
df_all.age.describe()

count    158681.000000
mean         47.145310
std         142.629468
min           1.000000
25%          28.000000
50%          33.000000
75%          42.000000
max        2014.000000
Name: age, dtype: float64

In [179]:
df_all['age']=df_all['age'].fillna(-1)

In [180]:
mask = df_all.age < 14
column_name = 'age'
df_all.loc[mask, column_name] = -1

In [181]:
mask = df_all.age > 100
column_name = 'age'
df_all.loc[mask, column_name] = -1

In [182]:
#selRows = df_all[df_all['age'] < 14].index
#df_all = df_all.drop(selRows, axis=0)

In [183]:
#selRows = df_all[df_all['age'] > 100].index
#df_all = df_all.drop(selRows, axis=0)

In [184]:
df_all['dac']=df_all['date_account_created'].str.split('-')
df_all = df_all.join(pd.DataFrame(df_all['dac'].tolist()))
df_all.rename(columns={0: 'dac_year',1: 'dac_month',2: 'dac_day'}, inplace=True)
df_all=df_all.drop(['date_account_created','dac', 'date_first_booking'], axis=1)

In [185]:
df_all['tfa_year']=df_all['timestamp_first_active'].astype(str).str.slice(0,4)
df_all['tfa_month']=df_all['timestamp_first_active'].astype(str).str.slice(4,6)
df_all['tfa_day']=df_all['timestamp_first_active'].astype(str).str.slice(6,8)
df_all=df_all.drop(['timestamp_first_active'], axis=1)

In [186]:
df_all['first_affiliate_tracked']=df_all['first_affiliate_tracked'].astype(str)

In [187]:
encoder = OneHotEncoder(['gender', 'signup_method', 'signup_flow', 'language', 'first_affiliate_tracked', 'affiliate_channel', 'affiliate_provider', 'signup_app', 'first_device_type', 'first_browser'])

In [188]:
encoder.fit(df_all)
df_combine=encoder.transform(df_all)

In [189]:
pd.set_option('display.max_rows', 500)

In [190]:
df_combine.dtypes

affiliate_channel_api                     object
affiliate_channel_content                 object
affiliate_channel_direct                  object
affiliate_channel_other                   object
affiliate_channel_remarketing             object
affiliate_channel_sem-brand               object
affiliate_channel_sem-non-brand           object
affiliate_channel_seo                     object
affiliate_provider_baidu                  object
affiliate_provider_bing                   object
affiliate_provider_craigslist             object
affiliate_provider_daum                   object
affiliate_provider_direct                 object
affiliate_provider_email-marketing        object
affiliate_provider_facebook               object
affiliate_provider_facebook-open-graph    object
affiliate_provider_google                 object
affiliate_provider_gsp                    object
affiliate_provider_meetup                 object
affiliate_provider_naver                  object
affiliate_provider_o

In [191]:
df_all

,affiliate_channel,affiliate_provider,age,country_destination,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,dac_year,dac_month,dac_day,tfa_year,tfa_month,tfa_day
0,direct,direct,-1.0,NDF,untracked,Chrome,Mac Desktop,-unknown-,gxn3p5htnn,en,Web,0,facebook,2010,06,28,2009,03,19
0,direct,direct,35.0,NaN,untracked,Mobile Safari,iPhone,FEMALE,5uwns89zht,en,Moweb,0,facebook,2010,06,28,2014,07,01
1,seo,google,38.0,NDF,untracked,Chrome,Mac Desktop,MALE,820tgsjxq7,en,Web,0,facebook,2011,05,25,2009,05,23
1,direct,direct,-1.0,NaN,untracked,Mobile Safari,iPhone,-unknown-,jtl0dijy2j,en,Moweb,0,basic,2011,05,25,2014,07,01
2,direct,direct,56.0,US,untracked,IE,Windows Desktop,FEMALE,4ft3gnwmtx,en,Web,3,basic,2010,09,28,2009,06,09
2,direct,direct,-1.0,NaN,linked,Chrome,Windows Desktop,-unknown-,xx0ulgorjt,en,Web,0,basic,2010,09,28,2014,07,01
3,direct,direct,42.0,other,untracked,Firefox,Mac Desktop,FEMALE,bjjt8pjhuk,en,Web,0,facebook,2011,12,05,2009,10,31
3,direct,direct,-1.0,NaN,linked,IE,Windows Desktop,-unknown-,6c6puo6ix0,en,Web,0,basic,2011,12,05,2014,07,01
4,direct,direct,41.0,US,untracked,Chrome,Mac Desktop,-unknown-,87mebub9p4,en,Web,0,basic,2010,09,14,2009,12,08
4,direct,direct,-1.0,NaN,untracked,Safari,Mac Desktop,-unknown-,czqhjk3yfe,en,Web,0,basic,2010,09,14,2014,07,01


In [192]:
mapping= {'NDF':0, 'US':1, 'other':2, 'FR':3, 'CA':4, 'GB':5, 'ES':6, 'IT':7, 'PT':8, 'NL':9, 'DE':10, 'AU':11}

Y_train=df_combine[df_combine['id'].isin(train['id'])]
Y_train=Y_train['country_destination'].map(mapping)
df_combine=df_combine.drop(['country_destination'],axis=1)

X_train=df_combine[df_combine['id'].isin(train['id'])]
X_test=df_combine[df_combine['id'].isin(test['id'])]
X_test=X_test.drop(['id'],axis=1)
X_train=X_train.drop(['id'],axis=1)
Y_train=pd.DataFrame(Y_train)

X_train=X_train.astype('float32', copy=False)
X_test=X_test.astype('float32', copy=False)
Y_train=Y_train.astype('int32', copy=False)

In [193]:
#XGBOOST

In [194]:
import xgboost as xgb
import time
import tensorflow as tf
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.33, random_state=5)

dtrain = xgb.DMatrix(x_train,y_train)
dtest = xgb.DMatrix(x_test)

# specify parameters via map

param = {'max_depth': 9, 'eta': 0.1, 'silent': 1, 'objective': 'multi:softmax',"num_class": 12, 'subsample' : 0.5, 'colsample_bytree' : 0.5,
               'seed' : 1,
               'eval_metric' : "merror",
               'nthread' : 3}
num_round = 25
training_start = time.perf_counter()
bst = xgb.train(param, dtrain, num_round)
training_end = time.perf_counter()
# make prediction
prediction_start = time.perf_counter()
preds = bst.predict(dtest)
prediction_end = time.perf_counter()
nd_y_test=y_test.values.reshape(len(y_test.values))
acc_xgb = (preds == nd_y_test).sum().astype(float) / len(preds)*100
xgb_train_time = training_end-training_start
xgb_prediction_time = prediction_end-prediction_start
print("XGBoost's prediction accuracy is: %3.2f" % (acc_xgb))
print("Time consumed for training: %4.3f" % (xgb_train_time))
print("Time consumed for prediction: %6.5f seconds" % (xgb_prediction_time))

XGBoost's prediction accuracy is: 63.40
Time consumed for training: 164.727
Time consumed for prediction: 1.93629 seconds


In [28]:
#Randomfrest

In [29]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
np.random.seed(0)

In [30]:
# Create a random forest Classifier. By convention, clf means 'Classifier'
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.33, random_state=7)
clf = RandomForestClassifier(n_jobs=100, random_state=0)
training_start = time.perf_counter()
clf.fit(x_train, y_train)
training_end = time.perf_counter()
prediction_start = time.perf_counter()
preds=clf.predict(x_test)
prediction_end = time.perf_counter()
nd_y_test=y_test.values.reshape(len(y_test.values))
acc_xgb = (preds == nd_y_test).sum().astype(float) / len(preds)*100
xgb_train_time = training_end-training_start
xgb_prediction_time = prediction_end-prediction_start
print("RF's prediction accuracy is: %3.2f" % (acc_xgb))
print("Time consumed for training: %4.3f" % (xgb_train_time))
print("Time consumed for prediction: %6.5f seconds" % (xgb_prediction_time))

C:\Python\Anaconda3-5.2\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


XGBoost's prediction accuracy is: 57.34
Time consumed for training: 2.212
Time consumed for prediction: 0.28585 seconds


In [117]:
#Softmax

In [118]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.33, random_state=7)

encoder = OneHotEncoder(['country_destination'])
encoder.fit(y_train)
y_train=encoder.transform(y_train)
y_train=y_train.astype('int32', copy=False)

In [120]:
import time
import os

def chk_processting_time(start_time, end_time):
    process_time = end_time - start_time
    p_time = int(process_time)
    p_min = p_time // 60
    p_sec = p_time %  60
    print('처리시간 : {p_min}분 {p_sec}초 경과되었습니다.'.format(
            p_min = p_min, 
            p_sec = p_sec
        ))
    return process_time

In [121]:
tf.reset_default_graph()
import tensorflow as tf
import random
import matplotlib.pyplot as plt

tf.set_random_seed(777)  # reproducibility

# parameters
learning_rate = 0.0001
training_epochs = 15
batch_size = 100
nb_classes = 12
keep_prob = 0.7
# input place holders
X = tf.placeholder(tf.float32, [None, 161])
Y = tf.placeholder(tf.int32, [None, 12])

# weights & bias for nn layers
# http://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
W1 = tf.get_variable("W1", shape=[161, 128],
                     initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([128]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob)

W2 = tf.get_variable("W2", shape=[128, 128],
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([128]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob)

W3 = tf.get_variable("W3", shape=[128, 128],
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([128]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob)

W4 = tf.get_variable("W4", shape=[128, 128],
                     initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([128]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob)

W5 = tf.get_variable("W5", shape=[128, 12],
                     initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([12]))
logits = tf.matmul(L4, W5) + b5
hypothesis = tf.nn.softmax(logits)


# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(y_train, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [122]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(2000):
    feed_dict = {X: x_train, Y: y_train}
    sess.run(optimizer, feed_dict=feed_dict)
    if step % 100 == 0:
        loss, acc = sess.run([cost, accuracy], feed_dict={X: x_train, Y: y_train})
        print("Step: {:5}, \t Loss: {:.3f}, \t Acc: {:.2%}".format(step, loss, acc))

Step:     0, 	 Loss: 2.533, 	 Acc: 8.51%
Step:   100, 	 Loss: 2.327, 	 Acc: 29.19%
Step:   200, 	 Loss: 2.326, 	 Acc: 29.25%
Step:   300, 	 Loss: 2.326, 	 Acc: 29.26%


KeyboardInterrupt: 

In [195]:
#제출용 파일생성
X_test=xgb.DMatrix(X_test)
result=bst.predict(X_test)
result=pd.DataFrame(result)
result=result[0]
inv_mapping= {0:'NDF', 1:'US', 2:'other', 3:'FR', 4:'CA', 5:'GB', 6:'ES', 7:'IT', 8:'PT', 9:'NL', 10:'DE', 11:'AU'}
result=result.map(inv_mapping)
result_test=test.drop(['date_account_created', 'timestamp_first_active',
       'date_first_booking', 'gender', 'age', 'signup_method', 'signup_flow',
       'language', 'affiliate_channel', 'affiliate_provider',
       'first_affiliate_tracked', 'signup_app', 'first_device_type',
       'first_browser'], axis=1)
result_test=result_test.join(result)
result_test.columns=['id','country']
filename='Submission_{}.csv'.format(acc_xgb) #Accracy 변수 설정
result_test.to_csv(filename,index=False)